# Minimize beam size and measure emittance on DIAG0

In [ ]:
# set up env for running on SLAC production servers
import os
os.environ['OMP_NUM_THREADS']=str(6)

In [ ]:
run_dir = '/home/physics/ml_tuning/20241114_DIAG0'

# Read pv info from YAML files

In [ ]:
import sys
import yaml
# sys.path.append("../../")
sys.path.append("../")

from common import get_pv_objects, save_reference_point, set_magnet_strengths, \
    measure_pvs

In [ ]:
pv_bounds = yaml.safe_load(open("../../pv_bounds.yml"))
pv_objects = get_pv_objects("../../tracked_pvs.yml")

In [ ]:
pv_bounds

In [ ]:
reference = yaml.safe_load(open("reference.yml"))

def reset_pvs():
    set_magnet_strengths(reference, pv_objects, validate=False)

In [ ]:
reference

In [ ]:
reset_pvs()

# Get model live rmats, design Twiss, and measurement quad length

In [ ]:
from meme.model import Model
import numpy as np

m = Model('SC_DIAG0', use_design=True)
magnet_length = m.get_zpos('DDG009A') - m.get_zpos('QDG009')
design_twiss = m.get_twiss('OTRDG02')

m = Model('SC_DIAG0', use_design=False)
rmat = m.get_rmat(from_device='DDG009A', to_device='OTRDG02')
rmat = np.stack([rmat[0:2, 0:2], rmat[2:4, 2:4]])

print('Magnet length:', magnet_length)
print('Design twiss:', design_twiss)
print('rmat:', rmat)

# Test screen measurement

In [5]:
from lcls_tools.common.devices.reader import create_screen, create_magnet
from lcls_tools.common.measurements.screen_profile import ScreenBeamProfileMeasurement

screen = create_screen(area='DIAG0', name='OTRDG02')
beamsize_measurement = ScreenBeamProfileMeasurement(device=screen)

In [ ]:
from matplotlib import pyplot as plt
results = beamsize_measurement.measure()
plt.imshow(results["raw_image"])
plt.imshow(results["processed_image"])

# Test magnet.scan() method

In [ ]:
magnet = create_magnet(area="DIAG0", name="QDG009")
magnet.metadata.length = magnet_length
scan_values = [-6.0, -3.0, -0.0] # placeholder values
magnet.scan(scan_settings=scan_values)

# Quad Scan Emittance

In [ ]:
energy = 3.0e9 # placeholder value
magnet
quad_scan = QuadScanEmittance(
    energy = energy, 
    magnet = magnet,
    scan_values = scan_values, 
    beamsize_measurement = beamsize_measurement,
    design_twiss = design_twiss,
    rmat = rmat,

)

result_dict = quad_scan.measure()

In [ ]:
result